In [48]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'gproject.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import re
import json
from bs4 import BeautifulSoup
import string
# Output directory
import spacy
from pathlib import Path
from spacy.training import Example
from spacy import displacy
from IPython.core.display import display, HTML
from IPython.display import clear_output
import requests

In [49]:
reading_path = '/Users/omerfaruk.davarci/Documents/MetabolysmAnalysisAPI/content2'
writing_path = '/Users/omerfaruk.davarci/Documents/MetabolysmAnalysisAPI/content4'

In [50]:
output_dir=Path(reading_path)

# Loading the model from the directory
print("Loading from", output_dir)
nlp = spacy.load(output_dir)

Loading from /Users/omerfaruk.davarci/Documents/MetabolysmAnalysisAPI/content2


In [51]:
import json

with open('annotated_data.json') as f:
    data = json.load(f)

In [89]:
from sklearn.model_selection import ShuffleSplit
import numpy as np
rs = ShuffleSplit(n_splits=1, test_size=.25, random_state=0)

train = []
test = []

for train_index, test_index in rs.split(data):
    train = np.array(data)[train_index.astype(int)]
    test = np.array(data)[test_index.astype(int)]
    
examples = []
for text, annots in data:
    examples.append(Example.from_dict(nlp.make_doc(text), annots))

In [90]:
ner = nlp.get_pipe('ner')

# Resume training
optimizer = nlp.resume_training()
#nlp.rehearse(examples, sgd=optimizer)

move_names = list(ner.move_names)
# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [91]:
# Importing requirements
from spacy.util import minibatch, compounding
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :

    sizes = compounding(1.0, 4.0, 1.001)
    # Training for 30 iterations     
    for itn in range(30):
    # shuffle examples before training
        random.shuffle(examples)
        # batch up the examples using spaCy's minibatch
        batches = minibatch(examples, size=sizes)
        losses = {}
        for batch in batches:
            # Calling update() over the iteration
            nlp.update(batch, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

Losses {'ner': 5.005305411510251}
Losses {'ner': 5.519683360853581}
Losses {'ner': 5.519683360853581}
Losses {'ner': 9.519658072484159}
Losses {'ner': 11.103299273306114}
Losses {'ner': 13.10316331328204}
Losses {'ner': 13.162434577720393}
Losses {'ner': 13.21696090787246}
Losses {'ner': 13.21791212994615}
Losses {'ner': 17.49003283761072}
Losses {'ner': 17.580578061179956}
Losses {'ner': 21.570648934728318}
Losses {'ner': 21.570648934728318}
Losses {'ner': 21.571582159309582}
Losses {'ner': 23.57158217958435}
Losses {'ner': 27.27190907555551}
Losses {'ner': 27.273009758142983}
Losses {'ner': 27.273057612774924}
Losses {'ner': 27.273057612774924}
Losses {'ner': 27.273057667360224}
Losses {'ner': 27.36085233599286}
Losses {'ner': 27.3610982924249}
Losses {'ner': 27.431136962243876}
Losses {'ner': 27.431136962243876}
Losses {'ner': 33.460191644815}
Losses {'ner': 33.46021060578814}
Losses {'ner': 33.46021068894643}
Losses {'ner': 37.423907898515246}
Losses {'ner': 37.423952193441856}
Los

ValueError: 

In [58]:
test_examples = []
for text, annots in test:
    test_examples.append(Example.from_dict(nlp.make_doc(text), annots))

print(len(test_examples))
print(len(data))

scores = nlp.evaluate(test_examples)

112
447


In [ ]:
# Output directory
from pathlib import Path
output_dir=Path(writing_path)
# Saving the model to the output directory
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta['name'] = 'pathway_ner_model'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)